# Minicurso Processamento de Linguagem Natural - Prática 1

Autores:
* Fernando Sola Pereira
* Eduardo Soares de Paiva

In [1]:
##########################################
# libs python
##########################################
import os
import re
import warnings

##########################################
# libs externas
##########################################
from gensim.models import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.test.utils import common_texts
from IPython.display import display, HTML, Latex, Markdown
import matplotlib.pyplot as plt
import nltk
from nltk.tokenize import sent_tokenize
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import TSNE

##########################################
# configurações
##########################################
warnings.filterwarnings('ignore')
sns.set_style("whitegrid")
pd.options.display.max_rows = 2000
pd.options.display.max_colwidth = 200

# faz com que o nltk baixe arquivos e modelos necessários para este notebook
nltk.download('punkt')
nltk.download('stopwords')

##########################################
# variáveis globais
##########################################
stopwords = nltk.corpus.stopwords.words('portuguese')

# Local utilizado para armazenar arquivos de dados e checkpoints de modelos 
# (altere de acordo com a sua necessidade).
# Por padrão supõe-se que está sendo executado no google colab e que 
# o google drive do usuário está acessível.
DATA_PATH = '/content/drive/MyDrive/Colab Notebooks/sbsi/data'

# semente fixa para garantir reprodutibilidade
DEFAULT_RANDOM_SEED = 42

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
if not os.path.exists(DATA_PATH):
  try:
    from google.colab import drive
    drive.mount('/content/drive')

    os.makedirs(DATA_PATH)
  except:
    print('Não está executando no ambiente Google Colab!')
else:
  print('Diretório existente!')

Mounted at /content/drive
Não está executando no ambiente Google Colab!


In [3]:
# Dataset (B2WReview 2018)
# https://www.kaggle.com/datasets/fredericods/ptbr-sentiment-analysis-datasets
# Versão reduzida e balanceada
df_review = pd.read_csv('https://docs.google.com/uc?export=download&id=1_EKfnjomkWks4VqTMIpcEIb6nB5P0Xz2')
df_review.columns = ['label','text']
df_review['label'] = df_review['label'].apply(lambda x: 1 if x == 'positivo' else 0)
df_review.head()

,label,text
0,1,"A entrega foi no prazo, as americanas estão de parabéns. A smart tv é muito boa, a navegação na internete e pelos aplicativos e excelente, não trava, sem falar da imagem que é de surpreender. reco..."
1,1,"Excelente produto, por fora em material acrílico super resistente e por dentro em adamantio, faz milagre com qualquer bebida. Sugiro aproveitarem a promoção antes que acabe."
2,1,"produto mto bom, com essa garrafinha vc pode até servir água pro megazord. To pensando em vender minha tv pra comprar 1 garrafa dessa. RECOMENDO"
3,1,O barulho e minimo e o vento é bem forte na velocidade 2
4,0,MEU PRODUTO NAO FOI ENTREGUE E A AMERICANAS ESTA DESCONTANDO NA FATURA DO MEU CARTÃO


In [4]:
def remover_caracteres_especiais(t):
  # substituir números por tokens
  t = re.sub(pattern=r'\bR[$][ ]*[\d]+[\d,.]*\b', repl=r'<dinheiro>', string=t).strip()
  t = re.sub(pattern=r'\b\d+\b', repl=r'<numero>', string=t).strip()
  # substituir caracteres especiais
  t = re.sub(r'[!,.?()\[\]\n]', r' ', t)
  return t

corpus = [remover_caracteres_especiais(s).split() for d in df_review['text'] for s in sent_tokenize(d)]

model = Word2Vec(sentences=corpus, size=100, window=5, min_count=5, iter=10)
model.save(os.path.join(DATA_PATH, "word2vec.model"))

In [5]:
model.wv.most_similar(['ótimo'], topn=20)

[('excelente', 0.8077907562255859),
 ('bom', 0.7625365257263184),
 ('perfeito', 0.7548098564147949),
 ('maravilhoso', 0.7436844110488892),
 ('otimo', 0.730481743812561),
 ('fantástico', 0.648720383644104),
 ('Ótimo', 0.6325582265853882),
 ('incrível', 0.6008961200714111),
 ('lindo', 0.5814872980117798),
 ('justo', 0.5725423097610474),
 ('Excelente', 0.5699535608291626),
 ('razoável', 0.5627008676528931),
 ('legal', 0.5607166290283203),
 ('bonito', 0.5371280908584595),
 ('acessível', 0.5370370745658875),
 ('sensacional', 0.5325623750686646),
 ('satisfatório', 0.530249297618866),
 ('espetacular', 0.5299338102340698),
 ('adequado', 0.5209857821464539),
 ('Otimo', 0.5166336297988892)]

In [6]:
model.wv.most_similar(['péssimo'], topn=20)

[('pessimo', 0.7945610880851746),
 ('horrível', 0.7473351955413818),
 ('Péssimo', 0.7135738134384155),
 ('Pessimo', 0.631472110748291),
 ('ruim', 0.6246446371078491),
 ('horrivel', 0.5440680980682373),
 ('orrivel', 0.5422359704971313),
 ('eletrônico', 0.5289033651351929),
 ('péssima', 0.5052193999290466),
 ('falso', 0.4744337797164917),
 ('ridículo', 0.4519907832145691),
 ('fraco', 0.44491952657699585),
 ('falsificado', 0.4435170292854309),
 ('pior', 0.44188541173934937),
 ('presta', 0.4333524703979492),
 ('ineficaz', 0.42788442969322205),
 ('primoroso', 0.42761000990867615),
 ('Horrível', 0.4150702655315399),
 ('central', 0.41432300209999084),
 ('inexistente', 0.41218751668930054)]

In [7]:
model.wv.similarity('ótimo', 'bom'), model.wv.similarity('ótimo', 'razoável'), model.wv.similarity('ótimo', 'ruim'), model.wv.similarity('ótimo', 'horrível')

(0.7625365, 0.5627008, 0.16449593, 0.15410893)

In [8]:
model.wv.most_similar(['caro'], topn=20)

[('barato', 0.7717796564102173),
 ('frete', 0.5797560811042786),
 ('reais', 0.5650491714477539),
 ('<dinheiro>', 0.5426352024078369),
 ('custa', 0.5292387008666992),
 ('cobram', 0.5137844085693359),
 ('pagar', 0.5094772577285767),
 ('Paguei', 0.5055925846099854),
 ('grátis', 0.49207910895347595),
 ('preço', 0.4911683201789856),
 ('compensa', 0.4740152060985565),
 ('gastar', 0.4668968915939331),
 ('salgado', 0.4666779637336731),
 ('caríssimo', 0.46633851528167725),
 ('paguei', 0.464053213596344),
 ('valor', 0.4630710482597351),
 ('cobrado', 0.4504176080226898),
 ('gratis', 0.4502483308315277),
 ('reias', 0.44913268089294434),
 ('barata', 0.4455825984477997)]

In [9]:
palavras = ['bom', 'regular', 'ruim', 'ótimo', 'otimo', 'razoável', 'pior', 'melhor', 'péssimo', 'excelente', 'maravilhoso', 'perfeito', 'impecável', 'horrível', 'pessimo']

palavras += [p[0] for p in model.wv.most_similar(['ótimo'], topn=2)]
palavras += [p[0] for p in model.wv.most_similar(['bom'], topn=2)]
palavras += [p[0] for p in model.wv.most_similar(['razoável'], topn=2)]
palavras += [p[0] for p in model.wv.most_similar(['péssimo'], topn=2)]
palavras += [p[0] for p in model.wv.most_similar(['ruim'], topn=2)]

palavras += ['barato','caro', 'pesado', '<dinheiro>', 'reais', 'custa', 'paga']

palavras = set(palavras)
palavrasa = np.array([model.wv.word_vec(p.strip()) for p in palavras])

mms = MinMaxScaler()
v_palavras = mms.fit_transform(palavrasa)
pca = PCA(n_components=2, random_state=DEFAULT_RANDOM_SEED)
v_palavras = pca.fit_transform(v_palavras)


import plotly.express as px
fig = px.scatter(x=v_palavras[:, 0], y=v_palavras[:, 1], text=[f' {p}' for p in palavras], template='seaborn')
fig.update_traces(textposition='top center')
fig.update_layout(
    height=700,
    title_text='Palavras relacionadas a sentimento e valor'
)
fig.show()


# fig, ax = plt.subplots(figsize=(16,9), dpi=90)

# # pontos
# plt.plot(v_palavras[:, 0], v_palavras[:, 1], marker='o', linestyle='')

# # textos
# for x, y, s in zip(v_palavras[:, 0], v_palavras[:, 1], palavras):
#   plt.text(x, y, '  {}'.format(s), fontsize=10)

# plt.show()

In [10]:
conj_a = set([a[0] for a in model.wv.most_similar(positive=['ruim', 'caro'], negative=['bom'], topn=20)])

In [11]:
conj_b = set([a[0] for a in model.wv.most_similar(positive=['pior', 'caro'], negative=['melhor'], topn=20)])

In [12]:
conj_c = set([a[0] for a in model.wv.most_similar(positive=['ótimo', 'caro'], negative=['péssimo'], topn=20)])

In [13]:
conj_a & conj_b & conj_c 

{'barato'}

In [14]:
documents = [TaggedDocument(remover_caracteres_especiais(s).split(), [i]) for i, doc in enumerate(df_review['text']) for s in sent_tokenize(doc)]
model = Doc2Vec(documents, vector_size=100, window=5, min_count=10, workers=4, epochs=20, seed=DEFAULT_RANDOM_SEED)

In [15]:
vx = [model.infer_vector(d, steps=20) for d in df_review['text'].apply(lambda x: remover_caracteres_especiais(x).split())]

In [28]:
frase = 'loja com atendimento excelente e o produto nem se fala parabéns'
vfrase = model.infer_vector(frase.split(), steps=50)

sim = cosine_similarity(vx, [vfrase])

df_resultado = df_review.copy()
df_resultado['Similaridade'] = sim
df_resultado.sort_values('Similaridade', ascending=False).head()

,label,text,Similaridade
77908,1,Loja com atendimento excelente e o produto nem se fala. Parabéns,0.769057
78709,1,Muito bom o produto parabéns ! Muito bom o produto parabéns ! Muito bom o produto parabéns !,0.719343
25000,1,O produto e de otima qualidade e a entrega nem se fala parabéns a lojas americanas.,0.715140
60215,1,"gostei muito do produto ótima qualidade, exatamente como descrito no site, parabéns.",0.694822
81173,1,"parabéns, ameiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii.",0.690712
